In [52]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
from scipy.interpolate import RegularGridInterpolator
import copy
import time
import opt_utils
import importlib
import trans_utils as T
importlib.reload(opt_utils)
importlib.reload(T)
from scipy.constants import g

In [101]:
scene_keypoints = np.array([[-0.25255756, 0.16720406 ,0.68192867],
 [-0.3970938 ,0.12950217 ,0.68181886],
 [ 0.5319556, 0.21342507,0.66371404],
 [-0.28037292,0.09597062 ,0.70344591],
 [-0.26905417 ,0.00379744 ,0.71206252],
 [ 1.10996314 ,-0.12611511,  0.62799189],
 [-0.23855288 ,-0.23952204 , 0.68195394]])

ee_pos = np.array([-0.4180, -0.1280,  0.9060])
ee_ori = np.array([ 0.4699,  0.5198, -0.4747,  0.5326]) #xyzw
ee_pose = np.concatenate([ee_pos, ee_ori])

# Assume same position pen and eef
scene_keypoints[5] = ee_pos
scene_keypoints[5][2] -= 0.05

keypoints = np.concatenate([[ee_pos], scene_keypoints], axis=0)
print(keypoints)

[[-0.418      -0.128       0.906     ]
 [-0.25255756  0.16720406  0.68192867]
 [-0.3970938   0.12950217  0.68181886]
 [ 0.5319556   0.21342507  0.66371404]
 [-0.28037292  0.09597062  0.70344591]
 [-0.26905417  0.00379744  0.71206252]
 [-0.418      -0.128       0.856     ]
 [-0.23855288 -0.23952204  0.68195394]]


In [102]:
keypoint_movable_mask = np.zeros(keypoints.shape[0], dtype=bool)
# ee
keypoint_movable_mask[0] = True
# pen
keypoint_movable_mask[5] = True

In [55]:
def objective(
        opt_vars, # eef pose in world frame
        og_bounds,
        keypoints_ee, # keypoints in ee frame
        keypoint_movable_mask,
        goal_constraints,
        path_constraints,
        init_pose_homo):
    
    opt_sol = opt_utils.unnormalize_vars(opt_vars, og_bounds)
    # Pose and Orientation
    opt_pose_homo = T.pose2mat([opt_sol[:3], T.euler2quat(opt_sol[3:6])])
    # Velocity
    opt_vel = opt_sol[6:]

    cost = 0.
    
    # Initial Cost
    init_pose_cost = 1.0 * opt_utils.consistency(opt_pose_homo[None], init_pose_homo[None], rot_weight=1.5)
    cost += init_pose_cost

    # subgoal constraint violation cost
    if goal_constraints is not None and len(goal_constraints) > 0:
        subgoal_constraint_cost = 0
        transformed_keypoints = opt_utils.transform_keypoints(opt_pose_homo, keypoints_ee, keypoint_movable_mask)
        subgoal_violation = []
        for constraint in goal_constraints:
            violation = constraint(transformed_keypoints[0], transformed_keypoints[1:], opt_vel)
            subgoal_violation.append(violation)
            subgoal_constraint_cost += np.clip(violation, 0, np.inf)
        subgoal_constraint_cost = 200.0*subgoal_constraint_cost
        cost += subgoal_constraint_cost
    
    # path constraint violation cost
    if path_constraints is not None and len(path_constraints) > 0:
        path_constraint_cost = 0
        transformed_keypoints = opt_utils.transform_keypoints(opt_pose_homo, keypoints_ee, keypoint_movable_mask)
        path_violation = []
        for constraint in path_constraints:
            violation = constraint(transformed_keypoints[0], transformed_keypoints[1:], opt_vel)
            path_violation.append(violation)
            path_constraint_cost += np.clip(violation, 0, np.inf)
        path_constraint_cost = 200.0*path_constraint_cost
        cost += path_constraint_cost
    return cost
    
    

In [114]:
ee_pose_homo = T.pose2mat([ee_pose[:3], ee_pose[3:]])
ee_pose_euler = np.concatenate([ee_pose[:3], T.quat2euler(ee_pose[3:])])
ee_pose_euler_vel = np.concatenate([ee_pose_euler, np.zeros(3)])

pos_bounds_min = [-0.45, -0.75, 0.698] #From config
pos_bounds_max = [0.2, 0.60, 1.2]
rot_bounds_min = np.array([-np.pi, -np.pi, -np.pi])  # euler angles
rot_bounds_max = np.array([np.pi, np.pi, np.pi])  # euler angles
vel_bounds_min = np.array([0.0, 0.0, 0.0])
vel_bounds_max = np.array([8.0, 0.1, 8.0])
og_bounds = [(b_min, b_max) for b_min, b_max in zip(np.concatenate([pos_bounds_min, rot_bounds_min, vel_bounds_min]), np.concatenate([pos_bounds_max, rot_bounds_max, vel_bounds_max]))]
bounds = [(-1, 1)] * len(og_bounds)




init_sol = opt_utils.normalize_vars(ee_pose_euler_vel, og_bounds)

# Center KeyPoints(world to ee frame)
centering_transform = np.linalg.inv(ee_pose_homo)
keypoints_ee = opt_utils.transform_keypoints(centering_transform, keypoints, keypoint_movable_mask)

In [104]:
### Stage 3 sub-goal constraints
def stage3_subgoal_constraint1(eef_pose, keypoints, eef_velocity):
    """Ensure the pen follows a parabolic trajectory to land in the trash bin."""
    dx = keypoints[5][0] - eef_pose[0]
    dy = keypoints[5][1] - eef_pose[1]
    dz = keypoints[5][2] - eef_pose[2]
    ds = np.sqrt(dx**2 + dy**2)
    v_horizontal = np.sqrt(eef_velocity[0]**2 + eef_velocity[1]**2) + 1e-6  # Add epsilon
    t = ds / v_horizontal
    z_predict = eef_pose[2] + eef_velocity[2] * t - 0.5 * g * t**2
    cost = (z_predict - keypoints[5][2])**2 - 1e-4  # Small tolerance delta
    return cost

def stage3_subgoal_constraint2(eef_pose, keypoints, eef_velocity):
    """Ensure the horizontal velocity vector is aligned with the target direction."""
    dx = keypoints[5][0] - eef_pose[0]
    dy = keypoints[5][1] - eef_pose[1]
    cost = (eef_velocity[0] * dy - eef_velocity[1] * dx)**2 - 1e-4  # Small tolerance delta
    return cost

def stage3_subgoal_constraint3(eef_pose, keypoints, eef_velocity):
    """Throw the pen at a 45-degree upward angle."""
    v_magnitude = np.sqrt(eef_velocity[0]**2 + eef_velocity[1]**2 + eef_velocity[2]**2) + 1e-6  # Add epsilon
    v_horizontal = np.sqrt(eef_velocity[0]**2 + eef_velocity[1]**2)
    cos_theta = v_horizontal / v_magnitude
    desired_cos_theta = np.sqrt(2) / 2  # cos(45 degrees)
    cost = (cos_theta - desired_cos_theta)**2 - 1e-4  # Small tolerance delta
    return cost


def stage3_path_constraint1(eef_pose, keypoints, eef_velocity):
    """The pen should keep aligned with the intended throwing direction during execution."""
    dx = keypoints[5][0] - eef_pose[0]
    dy = keypoints[5][1] - eef_pose[1]
    cost = (eef_velocity[0] * dy - eef_velocity[1] * dx)**2 - 1e-4  # Small tolerance delta
    return cost

goal_constraints = [stage3_subgoal_constraint1, stage3_subgoal_constraint2, stage3_subgoal_constraint3]
path_constraints = [stage3_path_constraint1]

In [115]:
aux_args = (og_bounds,
            keypoints_ee,
            keypoint_movable_mask,
            goal_constraints,
            path_constraints,
            ee_pose_homo)

In [116]:
global_opt_result = scipy.optimize.dual_annealing(
                func=objective,
                bounds=bounds,
                args=aux_args,
                maxfun=5000,
                x0=init_sol,
                no_local_search=False,
                minimizer_kwargs={
                    'method': 'SLSQP',
                    'options': {'maxiter': 200}
                },
            )

In [117]:
global_sol = opt_utils.unnormalize_vars(global_opt_result.x, og_bounds)
print(global_sol)

[-0.41802413 -0.12799517  0.86596932  0.37263224  1.55146303 -1.09030665
  6.81486977  0.04520776  6.90674672]


In [118]:
# Might need to update aux_args
gradient_opt_result = scipy.optimize.minimize(
                fun=objective,
                x0=global_sol,
                args=aux_args,
                bounds=bounds,
                method='SLSQP',
                options={'maxiter': 200},
            )

In [119]:
gradient_sol = opt_utils.unnormalize_vars(gradient_opt_result.x, og_bounds)
print(gradient_sol)

[-0.4175907  -0.12606892  0.86412664  3.14159265 -3.14159265 -3.14159265
  0.58536177  0.03727436  0.57318223]
